## Neural Networks for Patches

In [62]:
#global imports
import numpy as np
import tensorflow as tf
import tensorflow.contrib.learn as skflow
from sklearn.preprocessing import normalize
from sklearn.metrics import mean_squared_error, r2_score

In [16]:
from util import PatchesDataLoader, PatchesSKLearner

### Data Imports

In [4]:
data_path = '/home/jacky/ws/patches/data/'
pdl = PatchesDataLoader(0.25, data_path, [i for i in range(1)])

In [6]:
def one_hot(types, labels):
    result = np.zeros((labels.shape[0], len(types)))
    for i in range(labels.shape[0]):
        result[i][types[labels[i]]] = 1
    return result        

In [7]:
fc_tr_1h = one_hot([0, 1], pdl.labels['fc']['tr'])
fc_t_1h = one_hot([0, 1], pdl.labels['fc']['t'])

In [9]:
pfc_10_tr = pdl.labels['pfc_10']['tr'].reshape(-1, 1)
pfc_10_t = pdl.labels['pfc_10']['t'].reshape(-1, 1)

### DNN For Classification

In [11]:
batch_size = 100

In [14]:
clf = skflow.TensorFlowDNNClassifier(hidden_units=[500, 200], n_classes=2, batch_size=batch_size, steps=5000,
                                    optimizer="Adam", learning_rate=0.001, dropout=0.25, continue_training=True)

In [15]:
clf.fit(pdl.tr, pdl.labels['fc']['tr'])

Step #99, avg. train loss: 0.24250
Step #199, avg. train loss: 0.14069
Step #299, avg. train loss: 0.12051
Step #400, epoch #1, avg. train loss: 0.11671
Step #500, epoch #1, avg. train loss: 0.10248
Step #600, epoch #1, avg. train loss: 0.10545
Step #700, epoch #2, avg. train loss: 0.09329
Step #800, epoch #2, avg. train loss: 0.09174
Step #900, epoch #2, avg. train loss: 0.09603
Step #1000, epoch #2, avg. train loss: 0.08778
Step #1100, epoch #3, avg. train loss: 0.08880
Step #1200, epoch #3, avg. train loss: 0.08292
Step #1300, epoch #3, avg. train loss: 0.08670
Step #1400, epoch #4, avg. train loss: 0.08989
Step #1500, epoch #4, avg. train loss: 0.08192
Step #1600, epoch #4, avg. train loss: 0.08211
Step #1700, epoch #4, avg. train loss: 0.07573
Step #1800, epoch #5, avg. train loss: 0.06987
Step #1900, epoch #5, avg. train loss: 0.07815
Step #2000, epoch #5, avg. train loss: 0.07699
Step #2100, epoch #6, avg. train loss: 0.08023
Step #2200, epoch #6, avg. train loss: 0.07087
Step #

TensorFlowDNNClassifier(batch_size=100, class_weight=None, clip_gradients=5.0,
            config=None, continue_training=True, dropout=0.25,
            hidden_units=[500, 200], learning_rate=0.001, n_classes=2,
            optimizer='Adam', steps=5000, verbose=1)

In [17]:
PatchesSKLearner.print_accuracy(clf, pdl.tr, pdl.labels['fc']['tr'], pdl.t, pdl.labels['fc']['t'], "DNN Classification")

DNN Classification train accuracy 0.98483284592, test accuracy 0.974023739893


### DNN For Regression

In [18]:
reg = skflow.TensorFlowDNNRegressor(hidden_units=[500, 200], batch_size=batch_size, steps=5000,
                                    optimizer="Adam", learning_rate=0.001, dropout=0.25, continue_training=True)

In [20]:
reg.fit(pdl.tr, pdl.labels['pfc_10']['tr'])

Step #99, avg. train loss: 0.04443
Step #199, avg. train loss: 0.02253
Step #299, avg. train loss: 0.02157
Step #400, epoch #1, avg. train loss: 0.02044
Step #500, epoch #1, avg. train loss: 0.01968
Step #600, epoch #1, avg. train loss: 0.01879
Step #700, epoch #2, avg. train loss: 0.01895
Step #800, epoch #2, avg. train loss: 0.01840
Step #900, epoch #2, avg. train loss: 0.01765
Step #1000, epoch #2, avg. train loss: 0.01812
Step #1100, epoch #3, avg. train loss: 0.01744
Step #1200, epoch #3, avg. train loss: 0.01705
Step #1300, epoch #3, avg. train loss: 0.01742
Step #1400, epoch #4, avg. train loss: 0.01682
Step #1500, epoch #4, avg. train loss: 0.01630
Step #1600, epoch #4, avg. train loss: 0.01656
Step #1700, epoch #4, avg. train loss: 0.01677
Step #1800, epoch #5, avg. train loss: 0.01612
Step #1900, epoch #5, avg. train loss: 0.01612
Step #2000, epoch #5, avg. train loss: 0.01574
Step #2100, epoch #6, avg. train loss: 0.01591
Step #2200, epoch #6, avg. train loss: 0.01537
Step #

TensorFlowDNNRegressor(batch_size=100, clip_gradients=5.0, config=None,
            continue_training=True, dropout=0.25, hidden_units=[500, 200],
            learning_rate=0.001, n_classes=0, optimizer='Adam', steps=5000,
            verbose=1)

In [22]:
PatchesSKLearner.print_accuracy(reg, pdl.tr, pdl.labels['pfc_10']['tr'], pdl.t, pdl.labels['pfc_10']['t'], "DNN Regression PFC")

DNN Regression PFC train accuracy 0.775900502456, test accuracy 0.747592332814


In [23]:
PatchesSKLearner.print_mse(reg, pdl.tr, pdl.labels['pfc_10']['tr'], pdl.t, pdl.labels['pfc_10']['t'], "DNN Regression PFC mse")

DNN Regression PFC mse train mse 0.0228855780185, test mse 0.0254127871325


In [48]:
all_ferrari = np.r_[pdl.labels['ferrari']['tr'], pdl.labels['ferrari']['t']]
n_ferrari_tr = pdl.labels['ferrari']['tr'].shape[0]
ferrari_mean, ferrari_std = np.mean(all_ferrari), np.std(all_ferrari)
all_ferrari_normalized = (all_ferrari - ferrari_mean) / ferrari_std

In [50]:
ferrari_normalized_tr = all_ferrari_normalized[:n_ferrari_tr]
ferrari_normalized_t = all_ferrari_normalized[n_ferrari_tr:]

In [49]:
print np.mean(all_ferrari_normalized)

-5.62273625408e-17


In [51]:
reg_ferrari = skflow.TensorFlowDNNRegressor(hidden_units=[500, 200], batch_size=batch_size, steps=5000,
                                    optimizer="Adam", learning_rate=0.001, dropout=0.25, continue_training=True)

In [52]:
reg_ferrari.fit(pdl.tr, ferrari_normalized_tr)

Step #99, avg. train loss: 0.35687
Step #199, avg. train loss: 0.27995
Step #299, avg. train loss: 0.24340
Step #400, epoch #1, avg. train loss: 0.22318
Step #500, epoch #1, avg. train loss: 0.20634
Step #600, epoch #1, avg. train loss: 0.20706
Step #700, epoch #2, avg. train loss: 0.19558
Step #800, epoch #2, avg. train loss: 0.18454
Step #900, epoch #2, avg. train loss: 0.16887
Step #1000, epoch #2, avg. train loss: 0.15048
Step #1100, epoch #3, avg. train loss: 0.16090
Step #1200, epoch #3, avg. train loss: 0.15292
Step #1300, epoch #3, avg. train loss: 0.13893
Step #1400, epoch #4, avg. train loss: 0.14952
Step #1500, epoch #4, avg. train loss: 0.12793
Step #1600, epoch #4, avg. train loss: 0.13439
Step #1700, epoch #4, avg. train loss: 0.12794
Step #1800, epoch #5, avg. train loss: 0.09988
Step #1900, epoch #5, avg. train loss: 0.11308
Step #2000, epoch #5, avg. train loss: 0.11836
Step #2100, epoch #6, avg. train loss: 0.12537
Step #2200, epoch #6, avg. train loss: 0.11563
Step #

TensorFlowDNNRegressor(batch_size=100, clip_gradients=5.0, config=None,
            continue_training=True, dropout=0.25, hidden_units=[500, 200],
            learning_rate=0.001, n_classes=0, optimizer='Adam', steps=5000,
            verbose=1)

In [53]:
PatchesSKLearner.print_accuracy(reg_ferrari, pdl.tr, ferrari_normalized_tr, pdl.t, 
                                ferrari_normalized_t, "DNN Regression Ferrari Canny Normalized")

DNN Regression Ferrari Canny Normalized train accuracy 0.857218316759, test accuracy 0.834637472756


In [54]:
PatchesSKLearner.print_mse(reg_ferrari, pdl.tr, ferrari_normalized_tr, pdl.t, 
                                ferrari_normalized_t, "DNN Regression Ferrari Canny Normalized mse")

DNN Regression Ferrari Canny Normalized mse train mse 0.142164454541, test mse 0.167477717333


In [55]:
ferrari_preds_normalized_tr = reg_ferrari.predict(pdl.tr)
ferrari_preds_normalized_t = reg_ferrari.predict(pdl.t)

In [56]:
ferrari_preds_tr = (ferrari_preds_normalized_tr * ferrari_std) + ferrari_mean
ferrari_preds_t = (ferrari_preds_normalized_t * ferrari_std) + ferrari_mean

In [64]:
print "{0} train {1}, test {2}".format("DNN Regression Ferrari Canny", 
                                               r2_score(pdl.labels['ferrari']['tr'], ferrari_preds_tr), 
                                               r2_score(pdl.labels['ferrari']['t'], ferrari_preds_t))

DNN Regression Ferrari Canny train 0.857218318938, test 0.834637474128


In [60]:
print "{0} train mse {1}, test mse {2}".format("DNN Regression Ferrari Canny", 
                                               mean_squared_error(pdl.labels['ferrari']['tr'], ferrari_preds_tr), 
                                               mean_squared_error(pdl.labels['ferrari']['t'], ferrari_preds_t))

DNN Regression Ferrari Canny train mse 4.12614773566e-08, test mse 4.86083396298e-08
